In [6]:
# Import Required Packages
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import cv2
import os
import sys

print("All Required Packages Imported Successfully")

All Required Packages Imported Successfully


In [7]:
# Utilities for handling Depth Maps in PFM format
def read_pfm(file_path):
    """
    Read a PFM file into a numpy array.
    """
    with open(file_path, 'rb') as file:
        # Read header
        header = file.readline().decode('utf-8').strip()
        if header not in ['PF', 'Pf']:
            raise ValueError('Not a PFM file')
        
        # Read dimensions
        dim_line = file.readline().decode('utf-8').strip()
        width, height = map(int, dim_line.split())
        
        # Read scale/endianness
        scale_line = float(file.readline().decode('utf-8').strip())
        endian = '<' if scale_line < 0 else '>'
        
        # Read image data
        data = np.fromfile(file, endian + 'f')
        shape = (height, width) if header == 'Pf' else (height, width, 3)
        data = np.reshape(data, shape)
        
        # Flip array if scale is negative
        if scale_line < 0:
            data = np.flipud(data)
            
        return data

def write_pfm(file_path, image, scale=1):
    """
    Write a numpy array to a PFM file.
    """
    with open(file_path, 'wb') as file:
        # Write header
        header = 'Pf\n' if len(image.shape) == 2 else 'PF\n'
        file.write(header.encode('utf-8'))
        
        # Write dimensions
        file.write(f'{image.shape[1]} {image.shape[0]}\n'.encode('utf-8'))
        
        # Write scale/endianness
        endian = image.dtype.byteorder
        if endian == '<' or (endian == '=' and sys.byteorder == 'little'):
            scale = -scale
        file.write(f'{scale}\n'.encode('utf-8'))
        
        # Write image data
        image.tofile(file)

In [8]:
# Helper function to load depth maps
def load_depth_maps(pred_path, gt_path):
    """
    Load and preprocess predicted and ground truth depth maps
    """
    # Load depth maps based on file extension
    if pred_path.endswith('.pfm'):
        pred_depth = read_pfm(pred_path)
    else:
        pred_depth = cv2.imread(pred_path, cv2.IMREAD_ANYDEPTH)
        
    if gt_path.endswith('.pfm'):
        gt_depth = read_pfm(gt_path)
    else:
        gt_depth = cv2.imread(gt_path, cv2.IMREAD_ANYDEPTH)
    
    # Ensure same dimensions
    if pred_depth.shape != gt_depth.shape:
        pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]))
    
    return pred_depth, gt_depth

In [ ]:
# Start with basic quantitative metrics
def calculate_metrics(pred_depth, gt_depth, valid_mask=None):
    """
    Calculate common depth estimation metrics
    
    - RMSE
    - MAE
    - Relative Absolute Error
    - Relative Square Error
    - δ1
    - δ2
    - δ3
    - Correlation Coefficient
    """
    if valid_mask is None:
        valid_mask = gt_depth > 0  # Ignore zero depth pixels
        
    pred_valid = pred_depth[valid_mask]
    gt_valid = gt_depth[valid_mask]
    
    # Root Mean Square Error
    rmse = np.sqrt(mean_squared_error(gt_valid, pred_valid))
    
    # Mean Absolute Error
    mae = mean_absolute_error(gt_valid, pred_valid)
    
    # Relative Absolute Error
    rel_abs_err = np.mean(np.abs(pred_valid - gt_valid) / gt_valid)
    
    # Relative Square Error
    rel_sq_err = np.mean(((pred_valid - gt_valid) ** 2) / gt_valid)
    
    # δ1: Percentage of pixels where relative error is less than 1.25
    delta1 = np.mean(np.maximum(pred_valid / gt_valid, gt_valid / pred_valid) < 1.25)
    
    # δ2: Less than 1.25²
    delta2 = np.mean(np.maximum(pred_valid / gt_valid, gt_valid / pred_valid) < 1.25**2)
    
    # δ3: Less than 1.25³
    delta3 = np.mean(np.maximum(pred_valid / gt_valid, gt_valid / pred_valid) < 1.25**3)
    
    # Correlation coefficient
    correlation, _ = pearsonr(pred_valid, gt_valid)
    
    return {
        'RMSE': rmse,
        'MAE': mae,
        'Rel_Abs_Err': rel_abs_err,
        'Rel_Sq_Err': rel_sq_err,
        'Delta1': delta1,
        'Delta2': delta2,
        'Delta3': delta3,
        'Correlation': correlation
    }

In [ ]:
# Compare ground truth and predicted depth maps
def visualize_comparison(pred_depth, gt_depth, metrics, save_path=None):
    """
    Create visualization comparing predicted and ground truth depth maps
    """
    plt.figure(figsize=(15, 10))
    
    # Ground truth depth map
    plt.subplot(231)
    plt.imshow(gt_depth, cmap='viridis')
    plt.colorbar(label='Depth (m)')
    plt.title('Ground Truth Depth')
    
    # Predicted depth map
    plt.subplot(232)
    plt.imshow(pred_depth, cmap='viridis')
    plt.colorbar(label='Depth (m)')
    plt.title('Predicted Depth')
    
    # Error map
    error_map = np.abs(pred_depth - gt_depth)
    plt.subplot(233)
    plt.imshow(error_map, cmap='hot')
    plt.colorbar(label='Absolute Error (m)')
    plt.title('Error Map')
    
    # Scatter plot
    plt.subplot(234)
    valid_mask = gt_depth > 0
    plt.scatter(gt_depth[valid_mask], pred_depth[valid_mask], 
               alpha=0.1, s=1)
    max_depth = max(gt_depth.max(), pred_depth.max())
    plt.plot([0, max_depth], [0, max_depth], 'r--')
    plt.xlabel('Ground Truth Depth (m)')
    plt.ylabel('Predicted Depth (m)')
    plt.title('Depth Correlation')
    
    # Metrics text
    plt.subplot(235)
    plt.axis('off')
    metrics_text = '\n'.join([f'{k}: {v:.4f}' for k, v in metrics.items()])
    plt.text(0.1, 0.5, metrics_text, fontsize=10)
    plt.title('Metrics')
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()

In [ ]:
def evaluate_model_classic(pred_paths, gt_paths, output_dir=None):
    """
    Evaluate multiple depth maps and aggregate results
    """
    all_metrics = []
    
    for pred_path, gt_path in zip(pred_paths, gt_paths):
        # Load depth maps
        pred_depth, gt_depth = load_depth_maps(pred_path, gt_path)
        
        # Calculate metrics
        metrics = calculate_metrics(pred_depth, gt_depth)
        all_metrics.append(metrics)
        
        # Visualize if output directory is provided
        if output_dir:
            save_path = f"{output_dir}/comparison_{pred_path.split('/')[-1]}.png"
            visualize_comparison(pred_depth, gt_depth, metrics, save_path)
    
    # Calculate average metrics
    avg_metrics = {
        metric: np.mean([m[metric] for m in all_metrics])
        for metric in all_metrics[0].keys()
    }
    
    return avg_metrics, all_metrics

In [ ]:
# Perform Raycasting Evaluation on a depth map
def raycast_evaluation(depth_map, ray_origin, ray_direction, num_samples=100):
    """
    Perform raycasting evaluation on a depth map
    
    Args:
        depth_map: 2D numpy array of depth values
        ray_origin: (x, y, z) tuple for ray origin
        ray_direction: (dx, dy, dz) tuple for ray direction (normalized)
        num_samples: number of samples along the ray
    
    Returns:
        intersections: list of intersection points
        distances: list of distances to intersections
    """
    height, width = depth_map.shape
    
    # Create coordinate grids
    y, x = np.mgrid[0:height, 0:width]
    
    # Convert depth map to 3D points
    z = depth_map
    points = np.stack([x, y, z], axis=-1)
    
    # Sample points along the ray
    t = np.linspace(0, max(width, height), num_samples)
    ray_points = ray_origin + ray_direction[:, None] * t
    
    # Find intersections
    intersections = []
    distances = []
    
    for i in range(1, len(t)):
        p1 = ray_points[:, i-1]
        p2 = ray_points[:, i]
        
        # Convert to pixel coordinates
        x1, y1, z1 = p1
        x2, y2, z2 = p2
        
        # Check if ray segment intersects with depth surface
        if (0 <= x1 < width and 0 <= y1 < height and
            0 <= x2 < width and 0 <= y2 < height):
            
            # Get depth at these points
            d1 = depth_map[int(y1), int(x1)]
            d2 = depth_map[int(y2), int(x2)]
            
            # Check for intersection
            if (z1 - d1) * (z2 - d2) <= 0:
                # Linear interpolation to find intersection
                t_intersect = (d1 - z1) / (z2 - z1 - (d2 - d1))
                intersection = p1 + t_intersect * (p2 - p1)
                
                intersections.append(intersection)
                distances.append(np.linalg.norm(intersection - ray_origin))
    
    return np.array(intersections), np.array(distances)

In [ ]:
# Compare predicted and ground truth depth maps using raycasting
def compare_raycasting(pred_depth, gt_depth, num_rays=100):
    """
    Compare predicted and ground truth depth maps using raycasting
    """
    height, width = pred_depth.shape
    
    # Generate random rays
    ray_origins = np.random.rand(num_rays, 3) * np.array([width, height, 0])
    ray_directions = np.random.rand(num_rays, 3)
    ray_directions = ray_directions / np.linalg.norm(ray_directions, axis=1)[:, None]
    
    pred_errors = []
    gt_errors = []
    
    for i in range(num_rays):
        # Raycast both depth maps
        pred_intersections, pred_distances = raycast_evaluation(
            pred_depth, ray_origins[i], ray_directions[i])
        gt_intersections, gt_distances = raycast_evaluation(
            gt_depth, ray_origins[i], ray_directions[i])
        
        # Compare nearest intersections
        if len(pred_distances) > 0 and len(gt_distances) > 0:
            pred_nearest = np.min(pred_distances)
            gt_nearest = np.min(gt_distances)
            pred_errors.append(pred_nearest)
            gt_errors.append(gt_nearest)
    
    return np.array(pred_errors), np.array(gt_errors)

In [ ]:
# Analog of evaluate_model_classic but for the raycasting depth map evaluation
def evaluate_model_raycasting(pred_paths, gt_paths, output_dir=None, use_raycast=True):
    """
    Evaluate multiple depth maps and aggregate results
    """
    all_metrics = []
    raycast_errors = []
    
    for pred_path, gt_path in zip(pred_paths, gt_paths):
        # Load depth maps
        pred_depth, gt_depth = load_depth_maps(pred_path, gt_path)
        
        # Calculate standard metrics
        metrics = calculate_metrics(pred_depth, gt_depth)
        all_metrics.append(metrics)
        
        # Perform raycasting evaluation if requested
        if use_raycast:
            pred_rays, gt_rays = compare_raycasting(pred_depth, gt_depth)
            ray_error = np.mean(np.abs(pred_rays - gt_rays))
            raycast_errors.append(ray_error)
            metrics['Raycast_Error'] = ray_error
        
        # Visualize if output directory is provided
        if output_dir:
            save_path = os.path.join(output_dir, 
                                   f"comparison_{os.path.basename(pred_path)}.png")
            visualize_comparison(pred_depth, gt_depth, metrics, save_path)
    
    # Calculate average metrics
    avg_metrics = {
        metric: np.mean([m[metric] for m in all_metrics])
        for metric in all_metrics[0].keys()
    }
    
    if use_raycast:
        avg_metrics['Avg_Raycast_Error'] = np.mean(raycast_errors)
    
    return avg_metrics, all_metrics

In [ ]:
# TODO: Replace the predicted and ground truth depth maps with local paths on the Workstation
pred_paths = ['pred_depth_1.pfm', 'pred_depth_2.pfm']
gt_paths = ['gt_depth_1.pfm', 'gt_depth_2.pfm']
output_dir = '/evaluation_results'

# Run evaluation on classic metrics
avg_metrics, all_metrics = evaluate_model_classic(
    pred_paths, gt_paths, output_dir, use_raycast=True)

# Run evaluation on raycasting
avg_metrics_raycast, all_metrics_raycast = evaluate_model_raycasting(
    pred_paths, gt_paths, output_dir, use_raycast=True)


print("\nAverage Metrics (Classic):")
for metric, value in avg_metrics.items():
    print(f"{metric}: {value:.4f}")
    
print("\nAverage Metrics (Raycasting):")
for metric, value in avg_metrics_raycast.items():
    print(f"{metric}: {value:.4f}")